# 협업 필터링

## 협업 필터링 개요
```
✅ 정의
협업 필터링은 사용자의 구매 패턴이나 평점을 가지고 다른 사람들의 구매 패턴, 평점을 통해서 추천 하는 방법. 
추가적인 사용자의 개인정보나 아이템의 정보가 없이도 추천할 수 있는게 큰 장점.

✅ 종류
1. 최근접 이웃기반
2. 잠재 요인기반

✅ 장점
- 도메인 지식이 필요하지 않음 
- 사용자의 새로운 흥미를 발견하기 좋음 
- 시작단계의 모델로 선택하기 좋음 (추가적인 문맥정보 등의 필요가 없음) 

✅ 단점
- 새로운 아이템에 대해서 다루기가 힘듬 
- side features(고객의 개인정보, 아이템의 추가정보)를 포함시키기 어려움
```

## Neighborhood based method
```
✅ 정의
Neighborhood based Collaborative Filtering 은 메모리 기반 알고리즘으로 협업 필터링을 위해 개발된 초기 알고리즘

✅ 알고리즘
1.  User-based collaborative filtering
사용자의 구매 패턴(평점)과 유사한 사용자를 찾아서 추천 리스트 생성

2.  Item-based collaborative filtering
특정 사용자가 준 점수간의 유사한 상품을 찾아서 추천 리스트 생성
```

```
✅ 장점
- 간단하고 직관적인 접근 방식 때문에 구현 및 디버그가 쉬움
- 특정 Item 을 추천하는 이유는 정당화하기 쉽고 Item 기반 방법의 해석 가능성이 두드러짐
- 추천 리스트에 새로운 item 과 user 가 추가되어도 상대적으로 안정적

✅ 단점
- User 기반 방법의 시간, 속도, 메모리가 많이 필요
- 희소성 때문에 제한된 범위가 있음
    - John 의 Top-K 에만 관심이 있음
    - John 과 비슷한 이웃중에서 아무도 해리포터를 평가하지 않으면, 
      John 의 해리포터에 대한 등급 예측을 제공할 수가 없음
```

## Latent Factor Collaborative Filtering
```
✅ 정의
잠재 요인 협업 필터링은 Rating Matrix 에서 빈 공간을 채우기 위해서 사용자와 상품을 잘 표현하는 차원(Latent Factor)을 찾는 방법. 
잘 알려진 행렬 분해는 추천 시스템에서 사용되는 협업 필터링 알고리즘의 한 종류.
행렬 분해 알고리즘은 사용자-아이템 상호 작용 행렬을 두 개의 저 차원 직사각형 행렬의 곱으로 분해하여 작동.
```

### Matrix Factorization Principles
```
✅ 원리
사용자의 잠재요인과 아이템의 잠재요인을 내적해서 평점 매트릭스를 계산
```

### SVD
```
✅ 정의
고유값 분해(eigen value Decomposition)와 같은 행렬을 대각화 하는 방법
```
$$R  =  UΣV^T$$
```
U : (m, m) 직교행렬 
V : (n, n) 직교행렬 
Σ : (m, n) 직사각 대각행렬
```

```
✅ 예시
Explict Feedback 된 형태의 4명의 유저에 대한 3개의 아이템에 대한 평점 Matrix
1.  User Latent 와 Item Latent 를 임의로 초기화
2.  Gradient Descent 진행
3.  모든 평점에 대해서 반복 (epoch : 1)
4.  2~3의 과정을 10번 반복 (epoch : 10)

✅ 한계
1. 데이터에 결측치가 없어야 함
2. 대부분의 현업 데이터는 Sparse한 데이터

✅ 장점
- 매우 유연한 모델로 다른 Loss function 을 사용할 수 있음
- parallelized 가 가능함

✅ 단점
- 수렴까지 속도가 매우 느림
```

### ALS
```
✅ 정의
기존의 SGD 가 두개의 행렬(User Latent, Item Latent)을 동시에 최적화하는 방법이라면,
ALS는 두 행렬 중 하나를 고정시키고 다른 하나의 행렬을 순차적으로 반복하면서 최적화 하는 방법. 
기존의 최적화 문제가 convex 형태로 바뀌기에 수렴된 행렬을 찾을 수 있다는 것이 장점.

✅ 알고리즘
1.  초기 아이템, 사용자 행렬을 초기화 
2.  아이템 행렬을 고정하고 사용자 행렬을 최적화 
3.  사용자 행렬을 고정하고 아이템 행렬을 최적화 
4.  위의 2, 3 과정을 반복

✅ 장점
- SGD보다 수렴속도가 빠름 
- parallelized가 가능함

✅ 단점
- 오직 Loss Squares 만 사용가능
```

## 알고리즘

In [5]:
import numpy as np
from tqdm import tqdm_notebook as tqdm

# Base code : https://github.com/mickeykedia/Matrix-Factorization-ALS/blob/master/ALS%20Python%20Implementation.py
class AlternatingLeastSquares():
    def __init__(self, R, k, reg_param, epochs, verbose=False):
        '''
        R : rating matrix
        k : latent parameter
        learning_rate : alpha on weight update
        reg_param : beta on weight update
        epochs : training epochs
        verbose : print status
        '''
        self._R = R
        self._num_users, self._num_items = R.shape
        self._k = k
        self._reg_param = reg_param
        self._epochs = epochs
        self._verbose = verbose
        
    def fit(self):
        # init latent features
        self._users = np.random.normal(size=(self._num_users, self._k))
        self._items = np.random.normal(size=(self._num_items, self._k))
        
        # train while epochs
        self._training_process = []
        self._user_error = 0
        self._item_error = 0
        for epoch in range(self._epochs):
            for i, Ri in enumerate(self._R):
                self._users[i] = self.user_latent(i, Ri)
                self._user_error = self.cost()
                
            for j, Rj in enumerate(self._R.T):
                self._items[j] = self.item_latent(j, Rj)
                self._item_error = self.cost()
                
            cost = self.cost()
            self._training_process.append((epoch, cost))
            
            # print status
            if self._verbose == True and ((epoch + 1) % 10 == 0):
                print('Iteration: %d ; cost = %.4f' % (epoch + 1, cost))
                
    def cost(self):
        '''
        compute root mean square error
        return : rmse cost
        '''
        xi, yi = self._R.nonzero()
        cost = 0
        for x, y in zip(xi, yi):
            cost += pow(self._R[x, y] - self.get_prediction(x, y), 2)
        return np.sqrt(cost / len(xi))
    
    def user_latent(self, i, Ri):
        '''
        i : user index
        Ri : Rating of user index i
        return : convergence value of user latent of i index
        '''
        du = np.linalg.solve(np.dot(self._items.T, np.dot(np.diag(Ri), self._items)) + self._reg_param * np.eye(self._k), 
                             np.dot(self._items.T, np.dot(np.diag(Ri), self._R[i].T))).T
        return du
        
    def item_latent(self, j, Rj):
        '''
        j : item index
        Rj : Rating of item index j
        return : convergence value of item latent of j index
        '''
        di = np.linalg.solve(np.dot(self._users.T, np.dot(np.diag(Rj), self._users)) + self._reg_param * np.eye(self._k),
                            np.dot(self._users.T, np.dot(np.diag(Rj), self._R[:, j])))
        return di
        
    def get_prediction(self, i, j):
        '''
        get predicted rating : user_i, item_j
        return : prediction of r_ij
        '''
        return self._users[i, :].dot(self._items[j, :].T)
        
    def get_complete_matrix(self):
        '''
        return : complete matrix R^
        '''
        return self._users.dot(self._items.T)
    
# run example
if __name__ == '__main__':
    # rating matrix - User X Item : (7 X 5) 
    R = np.array([
        [1, 0, 0, 1, 3],
        [2, 0, 3, 1, 1],
        [1, 2, 0, 5, 0],
        [1, 0, 0, 4, 4],
        [2, 1, 5, 4, 0],
        [5, 1, 5, 4, 0],
        [0, 0, 0, 1, 0],
    ])

In [6]:
als = AlternatingLeastSquares(R = R, reg_param = 0.01, epochs = 100, verbose = True, k = 3)
als.fit()

Iteration: 10 ; cost = 0.0200
Iteration: 20 ; cost = 0.0110
Iteration: 30 ; cost = 0.0081
Iteration: 40 ; cost = 0.0068
Iteration: 50 ; cost = 0.0060
Iteration: 60 ; cost = 0.0056
Iteration: 70 ; cost = 0.0053
Iteration: 80 ; cost = 0.0051
Iteration: 90 ; cost = 0.0050
Iteration: 100 ; cost = 0.0048


In [8]:
import numpy as np

# 과학적 표기법 대신 소수점 6자리까지 나타낸다.
np.set_printoptions(precision=6, suppress=True)

als.get_complete_matrix()

array([[ 1.011009,  0.530446, -0.655946,  1.002086,  2.995154],
       [ 2.006235, -0.003494,  2.99259 ,  1.001851,  0.993138],
       [ 0.994303,  1.996699,  1.147425,  4.998452,  2.442027],
       [ 1.011316,  1.820737, -0.60602 ,  3.998965,  4.000714],
       [ 2.004571,  1.000289,  5.000836,  3.999399, -0.10348 ],
       [ 4.996402,  1.002425,  5.000347,  4.000778,  5.633392],
       [-0.096739,  0.436091, -0.028043,  0.997799,  0.197001]])